In [1]:
import numpy as np
import tensorflow as tf

/Users/seungbaeji/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Goal

- TensorFlow의 핵심파트 파악
- TensorFlow의 학습흐름 파악

# TensorFlow

TensorFlow는 데이터 흐름 그래프를 사용하는 수치 연산용 오픈소스 소프트웨어 라이브러리.  
단일 API를 통해 데스크톱, 서버 또는 휴대기기에 장착된 하나 이상의 CPU 또는 GPU에 연산을 배포할 수 있다.  

TensorFlow는 **분리된 두 파트**로 이루어져 있으며, 각 파트의 역할은 아래와 같다.
- `tf.Graph`: TensorFlow program
- `tf.Session`: TensorFlow runtime

즉, `tf.Graph`로 computational graph를 생성하고, `tf.Session`으로 해당 graph를 실행한다.

## 1. Graph (TensorFlow Program)

Computational Graph를 의미하며, `Operation`과 `Tensor`로 표현된다.  

- `tf.Operation`: Graph에서 node들로 표현되며, `tensor`를 소비하거나 생성하는 연산을 나타낸다. 그리고 각 `operation`들은 고유한 이름을 갖는다.
- `tf.Tensor`: Graph에서 edge들로 표현되며, `graph`에서 **앞으로 흐를** 값를 나타낸다. TensorFlow의 함수 대부분은 `tf.Tensor`를 반환한다. 

`print()`함수로 `Operation`에서 출력될 `Tensor`를 확인할 수 있다.

**중요: `Tensor`는 값을 갖지 않으며, graph의 요소들을 다루는 역할을 맡는다.**

In [2]:
x = 10
a = tf.constant(x, dtype=tf.float32)  # Constant Operation
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b

print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


### 1-a) TensorBoard

TensorFlow에서 시각화를 담당하는 도구. Python으로 작성한 computation graph를 시각화 가능하다.

In [3]:
writer = tf.summary.FileWriter('./constant')
writer.add_graph(tf.get_default_graph())
writer.flush()

In [4]:
! ls ./constant

events.out.tfevents.1548665226.Seungbaes-MacBook-Pro.local


In [4]:
! tensorboard --logdir ./constant

/Users/seungbaeji/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W0128 16:21:18.585929 Reloader tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0128 16:21:18.585928 123145446924288 tf_logging.py:120] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
TensorBoard 1.12.0 at http://Seungbaes-MacBook-Pro.local:6006 (Press CTRL+C to quit)
^C


## 2. Session (TensorFlow Runtime)

`tf.Session` 객체를 통해서 tensor를 평가(evaluation)할 수 있다.  
Session은 TensorFlow의 runtime에 관한 상태들을 캡슐화 하였으며, `Operation`들을 실행 시킨다.

`tf.Session.run` method는 tuple이나 dict 형식의 입력도 다룰 수 있다.

In [12]:
with tf.Session() as sess:
    print(total.eval())  # tf.get_default_session().run(total)
    print(sess.run({'ab': (b, a)}))
    
sess2 = tf.Session().as_default()
sess3 = tf.Session()

print(sess2 == tf.get_default_session())
print(sess3 == tf.get_default_session())

14.0
{'ab': (4.0, 10.0)}
False
False


### 2-a) Tensor Values

`Tensor`는 TensorFlow에서 data의 기본단위이다. `Tensor`의 값은 원시값(primitive value)로 이루어져 있으며, `Tensor`의 값들은 다차원 구조를 이루고 있다. 

`Tensor`의 정보는 rank와 shape로 알 수 있다. Rank는 tensor의 차원을 나타내며, Shape은 tensor의 각 차원내 속한 값들의 갯수를 나타낸다.  

In [7]:
a1, a2 = 3, 5.0  # Scalar
b1, b2, b3 = [], [1,2,3], [3,4,5,6,7,8,9]  # List
c1, c2 = [[1,2,3], [4,5,6]], [[1,2,3,4,5,6], [4,5,6,7,8,9]]
d1 = np.arange(100).reshape(2,5,5,2)

In [8]:
with tf.Session() as sess:
    rank_a = sess.run([tf.rank(a1), tf.rank(a2)])
    rank_b = sess.run([tf.rank(b1), tf.rank(b2), tf.rank(b3)])
    rank_c = sess.run([tf.rank(c1), tf.rank(c2)])
    rank_d = sess.run([tf.rank(d1)])
    
    shape_a = sess.run([tf.shape(a1), tf.shape(a2)])
    shape_b = sess.run([tf.shape(b1), tf.shape(b2), tf.shape(b3)])
    shape_c = sess.run([tf.shape(c1), tf.shape(c2)])
    shape_d = sess.run([tf.shape(d1)])
    
print('Rank of a1: {} / Shape of a1: {}\n'
      'Rank of a2: {} / Shape of a2: {}'\
      .format(rank_a[0], shape_a[0], 
              rank_a[1], shape_a[1]), end='\n\n')

print('Rank of b1: {} / Shape of b1: {}\n'
      'Rank of b2: {} / Shape of b2: {}\n'
      'Rank of b3: {} / Shape of b3: {}'\
      .format(rank_b[0], shape_b[0],
              rank_b[1], shape_b[1],
              rank_b[2], shape_b[2]), end='\n\n')

print('Rank of c1: {} / Shape of c1: {}\n'
      'Rank of c2: {} / Shape of c2: {}'\
      .format(rank_c[0], shape_c[0], 
              rank_c[1], shape_c[1]), end='\n\n')

print('Rank of d1: {} / Shape of d1: {}'\
      .format(rank_d[0], shape_d[0]), end='\n\n')

Rank of a1: 0 / Shape of a1: []
Rank of a2: 0 / Shape of a2: []

Rank of b1: 1 / Shape of b1: [0]
Rank of b2: 1 / Shape of b2: [3]
Rank of b3: 1 / Shape of b3: [7]

Rank of c1: 2 / Shape of c1: [2 3]
Rank of c2: 2 / Shape of c2: [2 6]

Rank of d1: 4 / Shape of d1: [2 5 5 2]



각각의 `tf.Session.run`마다 `Tensor` 는 하나의 값을 표현할 수 있다.  
아래의 예시에서 각 `run`마다 각기 다른 값을 갖는것을 알 수 있다.

In [9]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2

with tf.Session() as sess:
    print(sess.run(vec))
    print(sess.run(vec))
    print(sess.run((vec, out1, out2)))  # vec has a same value

[0.25800514 0.14975977 0.77474844]
[0.15670967 0.07949984 0.35647643]
(array([0.39900303, 0.12925124, 0.44903004], dtype=float32), array([1.399003 , 1.1292512, 1.44903  ], dtype=float32), array([2.399003 , 2.1292512, 2.44903  ], dtype=float32))


## 3. Train

### 3-a) Feeding

`placeholder`를 이용하여, 외부에서 graph에 값을 입력할 수 있다.

In [15]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
# r = tf.constant(x)
z = x + y


with tf.Session() as sess:
    print(sess.run(z, {x: 3, y: 4.5})
    print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

SyntaxError: invalid syntax (<ipython-input-15-555b4b5bb637>, line 9)

`placeholder`는 간단한 실험을 할때 많이 사용이 된다.  
하지만 모델에 대용량 데이터를 공급하기 위해서는 `tf.data`가 선호된다. 

In [11]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]

slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

with tf.Session() as sess:
    print(sess.run(next_item))
    print(sess.run(next_item))

[0 1]
[2 3]


`Dataset`이 stateful operation과 연관되어 있다면, iterator를 반드시 초기화 시켜주어야 한다.

In [12]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()


with tf.Session() as sess:
    sess.run(iterator.initializer)
    
    print(sess.run(next_row))
    print(sess.run(next_row))

[-1.7450634   1.75796     0.43232232]
[-0.34077045 -0.6131716   0.48733407]


### 3-b) Layer

모델은 학습시 graph내의 값들을 변화 시킨다. `tf.layers`는 딥러닝 모델에서 학습가능한 파라미터를 graph에 추가할 수 있는 가장 선호되는 방법이다.  
`Variable`과 `Operation`을 묶은 객체이며, `Variable`을 초기화 시켜줘야 사용이 가능하다.

In [18]:
tf.reset_default_graph()  # Clear the default graph

x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)  # Fully connected layer
y = linear_model(x)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(y, {x: [[1,2,3],[4,5,6]]}))

[[-0.31474972]
 [-2.1558566 ]]


In [20]:
writer = tf.summary.FileWriter('./dense1')
writer.add_graph(tf.get_default_graph())
writer.flush()

In [21]:
! tensorboard --logdir ./dense1

/Users/seungbaeji/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 1.12.0 at http://Seungbaes-MacBook-Pro.local:6006 (Press CTRL+C to quit)
^C


In [22]:
tf.reset_default_graph()  # Clear the default graph

x = tf.placeholder(tf.float32, shape=[None, 3])

# Define a Dense layer
units = 1
with tf.variable_scope("Dense") as vs:
    W = tf.Variable(tf.random_normal([3, units]), name='weight')
    b = tf.Variable(tf.random_normal([units]), name='bias')
    y = tf.matmul(x, W) + b

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(y, {x: [[1,2,3],[4,5,6]]}))

[[ 0.34105033]
 [-0.68243164]]


In [23]:
writer = tf.summary.FileWriter('./dense2')
writer.add_graph(tf.get_default_graph())
writer.flush()

In [24]:
! tensorboard --logdir ./dense2

/Users/seungbaeji/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 1.12.0 at http://Seungbaes-MacBook-Pro.local:6006 (Press CTRL+C to quit)
^C


### 3-c) Training

모델을 학습시키기 위해서는 첫번째로 loss 함수를 정의해 주어야 한다. `tf.losses`에서는 일반적인 loss 함수들을 제공한다.
두번째로는 `optimizer`를 사용하여야 한다. `optimizer`는 loss를 최소화 시키기 위하여 `variable`을 조금씩 변화 시킨다. 그리고 `optimizer`마다 `variable`을 변화시키는 방식이 다르다.

In [28]:
# Reset graph
tf.reset_default_graph()

# Define the data
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

# Define the model
linear_model = tf.layers.Dense(units=1)

# Loss
y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

# Train
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# Initialize
init = tf.global_variables_initializer()

# TensorBoard
writer = tf.summary.FileWriter('./training')
writer.add_graph(tf.get_default_graph())
writer.flush()

# Run
sess = tf.Session()
sess.run(init)
for i in range(1000):
    _, loss_value = sess.run((train, loss))
    if i % 20 == 0:
        print(loss_value)

print(sess.run(y_pred))
sess.close()

1.4782574
0.11314367
0.09955392
0.088301525
0.07832148
0.06946937
0.06161776
0.05465359
0.048476506
0.042997554
0.03813787
0.0338274
0.030004127
0.026612991
0.023605132
0.020937227
0.018570846
0.016471928
0.014610231
0.012958949
0.011494287
0.010195189
0.009042894
0.008020846
0.007114317
0.0063102297
0.0055970303
0.0049644383
0.0044033565
0.003905666
0.0034642387
0.0030727047
0.002725414
0.0024173828
0.0021441584
0.0019018239
0.0016868782
0.0014962221
0.0013271159
0.0011771193
0.0010440818
0.0009260768
0.00082140515
0.0007285635
0.0006462174
0.0005731849
0.00050840067
0.00045093993
0.000399972
0.00035476778
[[-0.02864295]
 [-1.0138794 ]
 [-1.999116  ]
 [-2.9843524 ]]


In [25]:
! tensorboard --logdir=./training

/Users/seungbaeji/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 1.12.0 at http://Seungbaes-MacBook-Pro.local:6006 (Press CTRL+C to quit)
^C


# Summary

1. TensorFlow는 연산을 `graph`와 `session` 파트로 구분하여, 단일 API를 통해 다양한 장치에 연산을 배포할 수 있다. 
2. `Graph` 외부에서 `graph`에 데이터를 주입하기 위하여, `placeholder`와 `tf.data` 가 이용된다.
3. 딥러닝 모델에서 학습가능한 paramter는 `tf.layer`를 통해 쉽게 설정이 가능하다.
4. 학습을 시키기 위해서는 `loss`와 `optimizer`를 정의해 주어야 한다.

# Reference

1. [TensorFlow: low-level Intoroduction](https://www.tensorflow.org/guide/low_level_intro)